<a href="https://colab.research.google.com/github/buganart/BUGAN/blob/master/script_VAEGAN_ver3_voxelsize1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#mount google drive
from google.colab import output
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
#right click shared folder IRCMS_GAN_collaborative_database and "Add shortcut to Drive" to My drive
%cd drive/My Drive/IRCMS_GAN_collaborative_database/

#record paths to resources
data_path = "Research/Peter/Tree_3D_models_obj/obj_files/"
run_path = "Experiments/colab-treegan/"

# !ls

/content/drive/.shortcut-targets-by-id/1ylB2p6N0qQ-G4OsBuwcZ9C0tsqVu9ww4/IRCMS_GAN_collaborative_database


In [3]:
!pip install trimesh
!pip install wandb -q
output.clear()

In [4]:
import io
import os
import trimesh
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary
from torch.utils.data import DataLoader, TensorDataset
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

# Ignore excessive warnings
import logging
logging.propagate = False 
logging.getLogger().setLevel(logging.ERROR)

# WandB – Import the wandb library
import wandb

#wandb log

In [5]:
!wandb login
output.clear()

In [6]:
#id None to start a new run. For resuming run, put the id of the run below
id = "2zo0s38j"
resume = False
if id is None:
    id = wandb.util.generate_id()
else:
    resume = True

run = wandb.init(project="tree-gan", id=id, resume="allow", dir=run_path)
print("run id:" + str(wandb.run.id))
wandb.run.name = str(wandb.run.id)
wandb.watch_called = False
wandb.run.save_code = True

#tag should be added from the panel
# wandb.run.tags = ["dev"]
# wandb.run.notes = "test run VAEGAN with BCELoss on VAE"
wandb.run.group = "VAEGAN"

run id:2zo0s38j


Streaming file created twice in same run: Experiments/colab-treegan/wandb/run-20200811_041620-2zo0s38j/wandb-history.jsonl
Streaming file created twice in same run: Experiments/colab-treegan/wandb/run-20200811_041620-2zo0s38j/wandb-events.jsonl


In [7]:
#keep track of hyperparams
config = wandb.config
config.batch_size = 4
config.test_batch_size = 1
config.epochs = 1000
config.vae_lr = 0.0001
config.d_lr = 0.00003           
config.seed = 1234
config.log_interval = 20

#dataset

In [8]:
def mesh2arrayCentered(mesh, voxel_size = 1, array_length = 64):
    #given array length 64, voxel size 2, then output array size is [128,128,128]
    array_size = np.ceil(np.array([array_length, array_length, array_length]) / voxel_size).astype(int)
    vox_array = np.zeros(array_size, dtype=bool)    #tanh: voxel representation [-1,1], sigmoid: [0,1]
    #scale mesh extent to fit array_length
    max_length = np.max(np.array(mesh.extents))
    mesh = mesh.apply_transform(trimesh.transformations.scale_matrix((array_length-1)/max_length))  #now the extent is [array_length**3]
    v = mesh.voxelized(voxel_size)  #max voxel array length = array_length / voxel_size

    #find indices in the v.matrix to center it in vox_array
    indices = ((array_size - v.matrix.shape)/2).astype(int)
    vox_array[indices[0]:indices[0]+v.matrix.shape[0], indices[1]:indices[1]+v.matrix.shape[1], indices[2]:indices[2]+v.matrix.shape[2]] = v.matrix

    return vox_array

In [9]:
dataset_artifact = run.use_artifact("dataset-tree:full", type='dataset')
dir_dict = dataset_artifact.metadata['dir_dict']
artifact_dir = dataset_artifact.download()
print(dir_dict)

wandb: Downloading large artifact dataset-tree:full, 566.02MB. 216 files... Done. 33.6s


{'old': ['old_1.obj', 'old_2.obj', 'old_3.obj'], 'raft': ['raft_1_1.obj', 'raft_1_2.obj', 'raft_1_4.obj', 'raft_1_3.obj', 'raft_1_5.obj', 'raft_1_6.obj', 'raft_1_7.obj', 'raft_1_8.obj', 'raft_1_9.obj', 'raft_1_10.obj'], 'group': ['group_1_1.obj', 'group_1_2.obj', 'group_1_3.obj', 'group_1_4.obj', 'group_1_5.obj', 'group_1_6.obj', 'group_1_7.obj', 'group_1_8.obj', 'group_1_9.obj', 'group_1_10.obj'], 'leaning': ['leaning_1_1.obj', 'leaning_1_2.obj', 'leaning_1_3.obj', 'leaning_1_4.obj', 'leaning_1_5.obj', 'leaning_1_6.obj', 'leaning_1_7.obj', 'leaning_1_8.obj', 'leaning_1_9.obj', 'leaning_1_10.obj', 'leaning_2_1.obj', 'leaning_2_2.obj', 'leaning_2_3.obj', 'leaning_2_4.obj', 'leaning_2_5.obj', 'leaning_2_6.obj', 'leaning_2_8.obj', 'leaning_2_10.obj', 'leaning_2_7.obj', 'leaning_2_9.obj'], 'windswept': ['windswept_1_1.obj', 'windswept_1_2.obj', 'windswept_1_3.obj', 'windswept_1_4.obj', 'windswept_1_5.obj', 'windswept_1_6.obj', 'windswept_1_7.obj', 'windswept_1_8.obj', 'windswept_1_9.obj', 

In [10]:
dataset = []

for data_cat in dir_dict:
    filename_list = dir_dict[data_cat]
    for filename in filename_list:
        filename = artifact_dir + "/" + data_cat + "/" + filename
        m = trimesh.load(filename, force='mesh')
        array = mesh2arrayCentered(m)
        dataset.append(array)
dataset = np.array(dataset)

In [11]:
dataset = torch.tensor(dataset)
print(torch.unsqueeze(dataset, -1).shape)
tensor_dataset = TensorDataset(torch.unsqueeze(dataset, 1))

#TODO: augment data

dataloader = DataLoader(tensor_dataset, batch_size=config.batch_size)

torch.Size([216, 64, 64, 64, 1])


#model description

In [12]:
#input: 128-d noise vector
#output: (64,64,64) array with values in [0,1]

class Generator(nn.Module):
    def __init__(self, input_channel=128):
        super(Generator, self).__init__()
        self.input_channel = input_channel
        self.fc_channel = 8 #16
        self.fc_size = 4

        num_unit1 = self.fc_channel   
        num_unit2 = 32   #32
        num_unit3 = 32   #64
        num_unit4 = 64   #128
        num_unit5 = 128   #256
        num_unit6 = 128   #512
        self.gen_fc = nn.Linear(self.input_channel, num_unit1 * self.fc_size * self.fc_size * self.fc_size)
        self.gen = nn.Sequential(

            nn.ConvTranspose3d(num_unit1, num_unit2, 3, 1, padding = 1),
            nn.BatchNorm3d(num_unit2),
            nn.ReLU(True),
            nn.ConvTranspose3d(num_unit2, num_unit3, 3, 1, padding = 1),
            nn.BatchNorm3d(num_unit3),
            nn.ReLU(True),
            nn.ConvTranspose3d(num_unit3, num_unit3, 3, 1, padding = 1),
            nn.BatchNorm3d(num_unit3),
            nn.ReLU(True),
            nn.ConvTranspose3d(num_unit3, num_unit3, 3, 1, padding = 1),
            nn.BatchNorm3d(num_unit3),
            nn.ReLU(True),

            nn.Upsample(scale_factor=2, mode='trilinear'),


            nn.ConvTranspose3d(num_unit3, num_unit4, 3, 1, padding = 1),
            nn.BatchNorm3d(num_unit4),
            nn.ReLU(True),
            nn.ConvTranspose3d(num_unit4, num_unit4, 3, 1, padding = 1),
            nn.BatchNorm3d(num_unit4),
            nn.ReLU(True),
            nn.ConvTranspose3d(num_unit4, num_unit4, 3, 1, padding = 1),
            nn.BatchNorm3d(num_unit4),
            nn.ReLU(True),
            nn.ConvTranspose3d(num_unit4, num_unit4, 3, 1, padding = 1),
            nn.BatchNorm3d(num_unit4),
            nn.ReLU(True),

            nn.Upsample(scale_factor=2, mode='trilinear'),


            nn.ConvTranspose3d(num_unit4, num_unit5, 3, 1, padding = 1),
            nn.BatchNorm3d(num_unit5),
            nn.ReLU(True),
            nn.ConvTranspose3d(num_unit5, num_unit5, 3, 1, padding = 1),
            nn.BatchNorm3d(num_unit5),
            nn.ReLU(True),
            nn.ConvTranspose3d(num_unit5, num_unit6, 3, 1, padding = 1),
            nn.BatchNorm3d(num_unit6),
            nn.ReLU(True),
            nn.ConvTranspose3d(num_unit6, num_unit6, 3, 1, padding = 1),
            nn.BatchNorm3d(num_unit6),
            nn.ReLU(True),

            nn.Upsample(scale_factor=2, mode='trilinear'),

            nn.ConvTranspose3d(num_unit6, num_unit6, 3, 1, padding = 1),
            nn.BatchNorm3d(num_unit6),
            nn.ReLU(True),
            nn.ConvTranspose3d(num_unit6, num_unit6, 3, 1, padding = 1),
            nn.BatchNorm3d(num_unit6),
            nn.ReLU(True),
            nn.ConvTranspose3d(num_unit6, num_unit5, 3, 1, padding = 1),
            nn.BatchNorm3d(num_unit5),
            nn.ReLU(True),
            nn.ConvTranspose3d(num_unit5, num_unit5, 3, 1, padding = 1),
            nn.BatchNorm3d(num_unit5),
            nn.ReLU(True),

            nn.Upsample(scale_factor=2, mode='trilinear'),

            nn.ConvTranspose3d(num_unit5, num_unit4, 3, 1, padding = 1),
            nn.BatchNorm3d(num_unit4),
            nn.ReLU(True),
            nn.ConvTranspose3d(num_unit4, num_unit3, 3, 1, padding = 1),
            nn.BatchNorm3d(num_unit3),
            nn.ReLU(True),
            nn.ConvTranspose3d(num_unit3, num_unit2, 3, 1, padding = 1),
            nn.BatchNorm3d(num_unit2),
            nn.ReLU(True),
            nn.ConvTranspose3d(num_unit2, num_unit1, 3, 1, padding = 1),
            nn.BatchNorm3d(num_unit1),
            nn.ReLU(True),
            nn.ConvTranspose3d(num_unit1, 1, 3, 1, padding = 1),
            nn.Sigmoid()                #tanh: voxel representation [-1,1], sigmoid: [0,1]
        )

    def forward(self, x):

        x = self.gen_fc(x)
        x = x.view(x.shape[0], self.fc_channel, self.fc_size, self.fc_size, self.fc_size)
        x = self.gen(x)
        return x


class Discriminator(nn.Module):
    def __init__(self, output_channel=1):
        super(Discriminator, self).__init__()
        self.output_channel = output_channel

        num_unit1 = 8   #16
        num_unit2 = 16   #32
        num_unit3 = 32   #64
        num_unit4 = 32  #128
        num_unit5 = 64   #256
        num_unit6 = 64   #512
        
        self.dis = nn.Sequential(
            nn.Conv3d(1, num_unit1, 3, 1, padding = 1),
            nn.BatchNorm3d(num_unit1),
            nn.ReLU(True),
            nn.Conv3d(num_unit1, num_unit2, 3, 1, padding = 1),
            nn.BatchNorm3d(num_unit2),
            nn.ReLU(True),
            nn.Conv3d(num_unit2, num_unit3, 3, 1, padding = 1),
            nn.BatchNorm3d(num_unit3),
            nn.ReLU(True),

            nn.MaxPool3d((2, 2, 2)),

            nn.Conv3d(num_unit3, num_unit3, 3, 1, padding = 1),
            nn.BatchNorm3d(num_unit3),
            nn.ReLU(True),
            nn.Conv3d(num_unit3, num_unit4, 3, 1, padding = 1),
            nn.BatchNorm3d(num_unit4),
            nn.ReLU(True),

            nn.MaxPool3d((2, 2, 2)),

            nn.Conv3d(num_unit4, num_unit5, 3, 1, padding = 1),
            nn.BatchNorm3d(num_unit5),
            nn.ReLU(True),
            nn.Conv3d(num_unit5, num_unit5, 3, 1, padding = 1),
            nn.BatchNorm3d(num_unit5),
            nn.ReLU(True),
            nn.Conv3d(num_unit5, num_unit4, 3, 1, padding = 1),
            nn.BatchNorm3d(num_unit4),
            nn.ReLU(True),

            nn.MaxPool3d((2, 2, 2)),

            nn.Conv3d(num_unit4, num_unit3, 3, 1, padding = 1),
            nn.BatchNorm3d(num_unit3),
            nn.ReLU(True),            
            nn.Conv3d(num_unit3, num_unit2, 3, 1, padding = 1),
            nn.BatchNorm3d(num_unit2),
            nn.ReLU(True),

            nn.MaxPool3d((2, 2, 2)),


            nn.Conv3d(num_unit2, num_unit1, 3, 1, padding = 1),
            nn.BatchNorm3d(num_unit1),
            nn.ReLU(True),
            nn.Conv3d(num_unit1, num_unit1, 3, 1, padding = 1),
            nn.BatchNorm3d(num_unit1),
            nn.ReLU(True),
        )

        self.dis_fc1 = nn.Sequential(
            nn.Linear(num_unit1 * 4 * 4 * 4, 128),
            nn.ReLU(True)
        )
        self.dis_fc2 = nn.Sequential(
            nn.Linear(128, self.output_channel),
            nn.Sigmoid()
        )

    def forward(self, x):

        x = self.dis(x)
        x = x.view(x.shape[0], -1)
        fx = self.dis_fc1(x)
        x = self.dis_fc2(fx)
        return x, fx


class VAEGAN(nn.Module):
    def __init__(self):
        super(VAEGAN, self).__init__()
        #VAE
        self.vae_encoder = Discriminator()
        self.encoder_mean = nn.Linear(128, 64)
        self.encoder_logvar = nn.Linear(128, 64)
        self.vae_decoder = Generator(input_channel=64)
        #GAN
        self.discriminator = Discriminator()

    #reference: https://github.com/YixinChen-AI/CVAE-GAN-zoos-PyTorch-Beginner/blob/master/CVAE-GAN/CVAE-GAN.py
    def noise_reparameterize(self,mean,logvar):
        eps = torch.randn(mean.shape).to(device)
        z = mean + eps * torch.exp(logvar)
        return z

    def forward(self, x):\
        #VAE
        _, f = self.vae_encoder(x)
        x_mean = self.encoder_mean(f)
        x_logvar = self.encoder_logvar(f)
        x = self.noise_reparameterize(x_mean, x_logvar)
        x = self.vae_decoder(x)
        #classifier and discriminator
        x = self.discriminator(x)
        return x

In [13]:
V = VAEGAN().to(device)
summary(V, (1, 64, 64, 64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1        [-1, 8, 64, 64, 64]             224
       BatchNorm3d-2        [-1, 8, 64, 64, 64]              16
              ReLU-3        [-1, 8, 64, 64, 64]               0
            Conv3d-4       [-1, 16, 64, 64, 64]           3,472
       BatchNorm3d-5       [-1, 16, 64, 64, 64]              32
              ReLU-6       [-1, 16, 64, 64, 64]               0
            Conv3d-7       [-1, 32, 64, 64, 64]          13,856
       BatchNorm3d-8       [-1, 32, 64, 64, 64]              64
              ReLU-9       [-1, 32, 64, 64, 64]               0
        MaxPool3d-10       [-1, 32, 32, 32, 32]               0
           Conv3d-11       [-1, 32, 32, 32, 32]          27,680
      BatchNorm3d-12       [-1, 32, 32, 32, 32]              64
             ReLU-13       [-1, 32, 32, 32, 32]               0
           Conv3d-14       [-1, 32, 32,

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3121: UserWarning: Default upsampling behavior when mode=trilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


#functions for pytorch network

In [14]:
def netarray2indices(array):
    coord_list = []
    if len(array.shape) == 5:
        array = array[0][0]
    x,y,z = array.shape
    for i in range(x):
        for j in range(y):
            for k in range(z):
                if array[i,j,k] > 0.5:        #tanh: voxel representation [-1,1], sigmoid: [0,1]
                    coord_list.append([i,j,k])
    print(len(coord_list))
    if len(coord_list) == 0:
        return np.array([[0,0,0]])  #return at least one point to prevent wandb 3dobject error
    return np.array(coord_list)

# array should be 3d
def netarray2mesh(array):
    if len(array.shape) != 3:
        raise Exception("netarray2mesh: input array should be 3d")

    #convert to bool dtype
    array = array > 0.5
    #array all zero gives error
    if np.sum(array) == 0:
        array[0,0,0] = True
    voxelmesh = trimesh.voxel.base.VoxelGrid(trimesh.voxel.encoding.DenseEncoding(array)).marching_cubes
    voxelmeshfile = voxelmesh.export(file_type='obj')
    voxelmeshfile = wandb.Object3D(io.StringIO(voxelmeshfile),file_type='obj')

    return voxelmesh, voxelmeshfile

def train_vaegan_model(VAEGAN, dataloader):
    #
    num_data = len(dataloader.dataset)
    
    torch.save(VAEGAN, os.path.join(wandb.run.dir, 'VAEGAN_model.pth'))
    wandb.save(os.path.join(wandb.run.dir, 'VAEGAN_model.pth'))

    #start training
    VAEGAN.to(device)

    vae_encoder = VAEGAN.vae_encoder.to(device)
    encoder_mean = VAEGAN.encoder_mean.to(device)
    encoder_logvar = VAEGAN.encoder_logvar.to(device)
    vae_decoder = VAEGAN.vae_decoder.to(device)
    discriminator = VAEGAN.discriminator.to(device)
    components = [VAEGAN, vae_encoder, encoder_mean, encoder_logvar, vae_decoder, discriminator]

    #optimizer   
    vae_encoder_optimizer = optim.Adam(vae_encoder.parameters(), lr=config.vae_lr)
    encoder_mean_optimizer = optim.Adam(encoder_mean.parameters(), lr=config.vae_lr)
    encoder_logvar_optimizer = optim.Adam(encoder_logvar.parameters(), lr=config.vae_lr)
    vae_decoder_optimizer = optim.Adam(vae_decoder.parameters(), lr=config.vae_lr)
    discriminator_optimizer = optim.Adam(discriminator.parameters(), lr=config.d_lr)


    #log models
    wandb.watch(VAEGAN, log="all")

    d_losses = []
    vae_losses = []
    for epoch in range(config.epochs):
        for c in components:
            c.train()

        d_ep_loss = 0.
        vae_ep_loss = 0.

        for dataset_batch in dataloader:
            dataset_batch = dataset_batch[0]    #dataset_batch was a list: [array], so just take the array inside
            dataset_batch = dataset_batch.float().to(device)
            #####TODO
            dloss, vaeloss = compute_loss(components, dataset_batch)
            
            #optimize
            
            #optimize
            discriminator_optimizer.zero_grad()
            vae_encoder_optimizer.zero_grad()
            encoder_mean_optimizer.zero_grad()
            encoder_logvar_optimizer.zero_grad()
            vae_decoder_optimizer.zero_grad()

            dloss.backward(retain_graph=True)
            vaeloss.backward(retain_graph=True)
            
            discriminator_optimizer.step()
            vae_encoder_optimizer.step()
            encoder_mean_optimizer.step()
            encoder_logvar_optimizer.step()
            vae_decoder_optimizer.step()

            
          

            #record loss
            d_ep_loss += dloss.detach()  
            vae_ep_loss += vaeloss.detach()

        #after each epoch, average ep_loss, and thenrecord total loss and sample generated obj
        d_ep_loss = d_ep_loss / num_data
        vae_ep_loss = vae_ep_loss / num_data
        d_losses.append(d_ep_loss)
        vae_losses.append(vae_ep_loss)
        print("discriminator, epoch"+str(epoch)+" : "+str(d_ep_loss))
        print("VAE, epoch"+str(epoch)+" : "+str(vae_ep_loss))

        #save model if necessary
        if epoch % config.log_interval == 0:

            sample_tree_array = generate_tree(vae_decoder)[0][0]  #only 1 tree
            sample_tree_indices = netarray2indices(sample_tree_array)
            _, voxelmeshfile = netarray2mesh(sample_tree_array)

            wandb.log({
            "discriminator loss": d_ep_loss,
            "VAE loss": vae_ep_loss,
            "sample_tree_indices": sample_tree_indices,
            "sample_tree_voxelmesh": voxelmeshfile})

            save_model(VAEGAN)


        else:
            wandb.log({
            "discriminator loss": d_ep_loss,
            "VAE loss": vae_ep_loss})
    
    #training end, save model again
    sample_tree_array = generate_tree(vae_decoder)[0][0] #only 1 tree
    sample_tree_indices = netarray2indices(sample_tree_array)
    _, voxelmeshfile = netarray2mesh(sample_tree_array)

    wandb.log({
    "discriminator loss": d_ep_loss,
    "VAE loss": vae_ep_loss,
    "sample_tree_indices": sample_tree_indices,
    "sample_tree_voxelmesh": voxelmeshfile})
    save_model(VAEGAN)
    
    print(d_losses)
    print(vae_losses)

# this function calculate loss of the model, 
def compute_loss(components, dataset_batch):

    VAEGAN, vae_encoder, encoder_mean, encoder_logvar, vae_decoder, discriminator = components
    batch_size = dataset_batch.shape[0]

    #loss function
    criterion_label = nn.BCELoss(reduction='sum')
    criterion_reconstruct = lambda input,target : nn.BCELoss(reduction='none')(input,target).mean() * batch_size    #mean per element in each data, sum over batch   
        
    #labels
    real_label = torch.unsqueeze(torch.ones(batch_size),1).float().to(device) * 0.9     # one side label smoothing
    fake_label = torch.unsqueeze(torch.zeros(batch_size),1).float().to(device)

    ############
    #   discriminator (and classifier if necessary)
    ############
    
    #generate fake trees
    latent_size = vae_decoder.input_channel
    z = torch.randn(batch_size, latent_size).float().to(device) #noise vector
    tree_fake = vae_decoder(z)

    #fake data (data from generator) 
    dout_fake, _ = discriminator(tree_fake.clone().detach())   #detach so no update to generator
    dloss_fake = criterion_label(dout_fake, fake_label)
    #real data (data from dataloader)
    dout_real, _ = discriminator(dataset_batch)
    dloss_real = criterion_label(dout_real, real_label)

    ############
    #   VAE
    ############
    _, z = vae_encoder(dataset_batch)
    z_mean = encoder_mean(z)
    z_logvar = encoder_logvar(z)
    z = VAEGAN.noise_reparameterize(z_mean, z_logvar)

    reconstructed_data = vae_decoder(z)
    #!!!check if we need more than MSELoss
    vae_rec_loss = criterion_reconstruct(reconstructed_data, dataset_batch)    #loss is scaled to one

    #output of the vae should fool discriminator
    vae_out_d, _ = discriminator(reconstructed_data)
    vae_d_loss = criterion_label(vae_out_d, real_label)

    dloss = (dloss_fake + dloss_real) / 2   #scale the loss to one
    vae_loss = (vae_rec_loss + vae_d_loss) / 2   #scale the loss to one
    return dloss, vae_loss


def save_model(model, model_path = os.path.join(wandb.run.dir, 'model_dict.pth')):
    torch.save(model.state_dict(), model_path)
    wandb.save(model_path)

def load_model(model_path = 'model_dict.pth'):
    model = VAEGAN()

    model_file = wandb.restore(model_path)
    model.load_state_dict(torch.load(model_file.name))

    return model

def generate_tree(vae_decoder, num_trees = 1):
    
    #generate noise vector
    latent_size = vae_decoder.input_channel
    z = torch.randn(num_trees, latent_size).to(device) 
    tree_fake = vae_decoder(z)

    return tree_fake.detach().cpu().numpy()

#train

In [ ]:
# check if resume

if resume:
    vaegan = load_model()
else:
    vaegan = VAEGAN()

#set seed
torch.manual_seed(config.seed)
torch.autograd.set_detect_anomaly(True)

train_vaegan_model(vaegan, dataloader)        #if dataloader has only 1 tree, the training time is 72s per epoch.

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3121: UserWarning: Default upsampling behavior when mode=trilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


discriminator, epoch0 : tensor(0.2106, device='cuda:0')
VAE, epoch0 : tensor(0.2663, device='cuda:0')
15
discriminator, epoch1 : tensor(0.1645, device='cuda:0')
VAE, epoch1 : tensor(0.1949, device='cuda:0')
discriminator, epoch2 : tensor(0.1656, device='cuda:0')
VAE, epoch2 : tensor(0.1943, device='cuda:0')
discriminator, epoch3 : tensor(0.1639, device='cuda:0')
VAE, epoch3 : tensor(0.1932, device='cuda:0')
discriminator, epoch4 : tensor(0.1677, device='cuda:0')
VAE, epoch4 : tensor(0.1957, device='cuda:0')
discriminator, epoch5 : tensor(0.1713, device='cuda:0')
VAE, epoch5 : tensor(0.1963, device='cuda:0')
discriminator, epoch6 : tensor(0.1828, device='cuda:0')
VAE, epoch6 : tensor(0.2038, device='cuda:0')
discriminator, epoch7 : tensor(0.1662, device='cuda:0')
VAE, epoch7 : tensor(0.1957, device='cuda:0')
discriminator, epoch8 : tensor(0.2080, device='cuda:0')
VAE, epoch8 : tensor(0.2100, device='cuda:0')
discriminator, epoch9 : tensor(0.1767, device='cuda:0')
VAE, epoch9 : tensor(0.